# Getting Started

To get started with this Jupyter notebook, you will need to either:

- Install [Anaconda](https://www.anaconda.com/)
- Install [Jupyter Lab](https://jupyter.org/install)
- Install [Visual Studio Code with the Jupyter Notebook Extension](https://code.visualstudio.com/)

Once you've configured your frameowrk of choice, you will then need to install the following Python libraries (`pandas`, `requests`) by running the cell below.

Thereafter, navigate to the cell below the heading `Configure your data query` and modify the username to the one you were provided by the M2C2 Team. You will be prompted for your password each time you login.

In [ ]:
! pip install requests
! pip install pandas

## Load libraries and custom functions

### Note: DO NOT modify any of the functions below

In [1]:
import urllib.parse
import requests
import pandas as pd
from getpass import getpass

In [2]:
def get_m2c2kit_access_token(username=None, password=None):
    # specify login endpoint URL
    login_url = "https://prod.m2c2kit.com/auth/token"
    payload = f"=grant_type%3D&=scope%3D&=client_id%3D&=client_secret%3D&username={username}&password={password}"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded"
    }

    # attempt login
    login_response = requests.request("POST", login_url, data=payload, headers=headers)
    access_token = login_response.json().get("access_token")
    return access_token

In [3]:
def get_m2c2kit_trial_level_data(access_token=None, study_id=None, start_date=None, end_date=None, activity_name=None):

    # check if required fields present
    if access_token is None:
        raise ValueError("access_token is required")
    if study_id is None:
        raise ValueError("study_id is required")
    if start_date is None:
        raise ValueError("start_date is required")
    if end_date is None:
        raise ValueError("end_date is required")
    if activity_name is None:
        raise ValueError("activity_name is required")

    # specify query endpoint URL
    query_url = "https://prod.m2c2kit.com/query/"

    # specify query parameters ----
    querystring = {"fields":"study_uid,uid,session_uid,activity_name,event_type,content,metadata",
                "activity_name":activity_name,
                "format":"json",
                "study_uid":study_id,
                "start_date":start_date,
                "end_date":end_date,
                "skip":"0",
                "limit":"1000"}

    payload = ""
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {access_token}"
    }

    # TODO: check for total and run with new limit and skip if reached limit ----
    data_response = requests.request("GET", query_url, data=payload, headers=headers, params=querystring)
    data_json = data_response.json()
    data_records = data_json.get("results")
    data_total = data_json.get("total")
    data_limit = data_json.get("limit")
    data_df = pd.DataFrame(data_records)

    # iterate over the dataset to get all trials ----
    all_trials = []
    for index, row in data_df.iterrows():
        json_data = row['content'].get("trials", [])
        all_trials.extend(json_data)

    # convert all trials to dataframe ----
    df_all = pd.DataFrame(all_trials)
    return df_all, data_total, data_limit

## Configure your data query

In [5]:
# specify parameters for M2C2kit backend
backend_username = "[enter username here]"
backend_password = getpass('Enter password for M2C2kit backend...') # you will be prompted for a password

# login to M2C2kit backend to get access token for querying data (expires in X minutes)
access_token = get_m2c2kit_access_token(username=backend_username, 
                                password=backend_password)


# query range of data for a given study
study_id = "demo"
start_date = "2023-10-18"
end_date = "2023-10-20"


# query Symbol Search activity data
df_symbolsearch, total_symbolsearch, limit_symbolsearch = get_m2c2kit_trial_level_data(access_token=access_token, 
                                                             study_id=study_id, 
                                                             start_date=start_date, end_date=end_date, 
                                                             activity_name="Symbol Search")

# query Symbol Search activity data
df_gridmemory, total_gridmemory, limit_gridmemory = get_m2c2kit_trial_level_data(access_token=access_token, 
                                                           study_id=study_id, 
                                                           start_date=start_date, end_date=end_date, 
                                                           activity_name="Grid Memory")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
display(df_symbolsearch.head(3))

,document_uuid,session_uuid,activity_uuid,activity_id,activity_version,device_timezone,device_timezone_offset_minutes,activity_begin_iso8601_timestamp,trial_begin_iso8601_timestamp,trial_index,...,user_response_index,correct_response_index,quit_button_pressed,device_metadata,study_id,session_id,participant_id,api_key,group,wave
0,f3f71685-ac2b-4156-b676-258d9d619c48,8e5cd9f4-7875-4048-8cf8-ca93974e2288,b7c6a8c4-cdbc-45d5-a7cb-b58ecb8712b0,symbol-search,0.8.4,America/New_York,240,2023-10-18T17:06:42.601Z,2023-10-18T17:06:49.187Z,0.0,...,1.0,1.0,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,a0770c06-7f16-4b56-882d-dc06ba662677,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
1,f3f71685-ac2b-4156-b676-258d9d619c48,8e5cd9f4-7875-4048-8cf8-ca93974e2288,b7c6a8c4-cdbc-45d5-a7cb-b58ecb8712b0,symbol-search,0.8.4,America/New_York,240,2023-10-18T17:06:42.601Z,2023-10-18T17:06:49.187Z,0.0,...,1.0,1.0,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,a0770c06-7f16-4b56-882d-dc06ba662677,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
2,77440d57-a74d-4985-90f0-dffa3246b158,8e5cd9f4-7875-4048-8cf8-ca93974e2288,b7c6a8c4-cdbc-45d5-a7cb-b58ecb8712b0,symbol-search,0.8.4,America/New_York,240,2023-10-18T17:06:42.601Z,2023-10-18T17:06:50.621Z,1.0,...,0.0,0.0,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,a0770c06-7f16-4b56-882d-dc06ba662677,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
3,f3f71685-ac2b-4156-b676-258d9d619c48,8e5cd9f4-7875-4048-8cf8-ca93974e2288,b7c6a8c4-cdbc-45d5-a7cb-b58ecb8712b0,symbol-search,0.8.4,America/New_York,240,2023-10-18T17:06:42.601Z,2023-10-18T17:06:49.187Z,0.0,...,1.0,1.0,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,a0770c06-7f16-4b56-882d-dc06ba662677,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
4,77440d57-a74d-4985-90f0-dffa3246b158,8e5cd9f4-7875-4048-8cf8-ca93974e2288,b7c6a8c4-cdbc-45d5-a7cb-b58ecb8712b0,symbol-search,0.8.4,America/New_York,240,2023-10-18T17:06:42.601Z,2023-10-18T17:06:50.621Z,1.0,...,0.0,0.0,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,a0770c06-7f16-4b56-882d-dc06ba662677,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,bde7b6b4-0dcf-42ce-bd6d-df62d577de7a,9f4f5328-50bc-4fbb-b98f-327d3780042c,a3d9ce7a-7fba-4f81-9ada-9ed98b2ecd46,symbol-search,0.8.4,America/New_York,240,2023-10-19T18:04:03.679Z,2023-10-19T18:04:09.786Z,0.0,...,1.0,1.0,False,{'userAgent': 'Mozilla/5.0 (iPhone; CPU iPhone...,PSAC_PILOT,0b9db820-8c6f-408f-8d2a-6fd3a2b18b8e,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
57,26b0fd49-e9a6-4fe7-a5f7-3b6c704b6d8c,9f4f5328-50bc-4fbb-b98f-327d3780042c,a3d9ce7a-7fba-4f81-9ada-9ed98b2ecd46,symbol-search,0.8.4,America/New_York,240,2023-10-19T18:04:03.679Z,2023-10-19T18:04:11.029Z,1.0,...,0.0,0.0,False,{'userAgent': 'Mozilla/5.0 (iPhone; CPU iPhone...,PSAC_PILOT,0b9db820-8c6f-408f-8d2a-6fd3a2b18b8e,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
58,bde7b6b4-0dcf-42ce-bd6d-df62d577de7a,9f4f5328-50bc-4fbb-b98f-327d3780042c,a3d9ce7a-7fba-4f81-9ada-9ed98b2ecd46,symbol-search,0.8.4,America/New_York,240,2023-10-19T18:04:03.679Z,2023-10-19T18:04:09.786Z,0.0,...,1.0,1.0,False,{'userAgent': 'Mozilla/5.0 (iPhone; CPU iPhone...,PSAC_PILOT,0b9db820-8c6f-408f-8d2a-6fd3a2b18b8e,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
59,26b0fd49-e9a6-4fe7-a5f7-3b6c704b6d8c,9f4f5328-50bc-4fbb-b98f-327d3780042c,a3d9ce7a-7fba-4f81-9ada-9ed98b2ecd46,symbol-search,0.8.4,America/New_York,240,2023-10-19T18:04:03.679Z,2023-10-19T18:04:11.029Z,1.0,...,0.0,0.0,False,{'userAgent': 'Mozilla/5.0 (iPhone; CPU iPhone...,PSAC_PILOT,0b9db820-8c6f-408f-8d2a-6fd3a2b18b8e,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None


In [78]:
display(df_gridmemory.head(3))

,document_uuid,session_uuid,activity_uuid,activity_id,activity_version,device_timezone,device_timezone_offset_minutes,activity_begin_iso8601_timestamp,trial_begin_iso8601_timestamp,trial_index,...,user_interference_actions,number_of_correct_dots,quit_button_pressed,device_metadata,study_id,session_id,participant_id,api_key,group,wave
0,6ecaf117-f2d8-4bc4-ba23-167023b2536f,d1017f29-5395-4437-80be-068c7a684737,1baa48fc-0bc7-4bba-8884-32415838058b,grid-memory,0.8.4,America/New_York,240,2023-10-18T17:06:58.583Z,2023-10-18T17:07:05.419Z,0,...,"[{'elapsed_duration_ms': 800.8999999761581, 'a...",2,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,a0770c06-7f16-4b56-882d-dc06ba662677,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
1,824b5a77-01bc-476d-92d9-612c9f18cdd4,411813e9-ceff-40b4-8e67-8aa04db3ba44,8841ee49-29e0-451f-827b-8e66ab9b95ff,grid-memory,0.8.4,America/New_York,240,2023-10-18T17:08:54.929Z,2023-10-18T17:09:04.781Z,0,...,"[{'elapsed_duration_ms': 2506.699999988079, 'a...",0,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,ab310dfb-110d-45e9-b9a2-7810b068193d,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
2,46f5760a-2d3c-48f0-98f1-a36c91d91241,8d4a137a-9a8e-45e9-aa1d-040016bb2b73,26fe417a-f0c1-4335-90ff-cbab8dfb4ac9,grid-memory,0.8.4,America/New_York,240,2023-10-18T17:11:28.761Z,2023-10-18T17:11:31.680Z,0,...,"[{'elapsed_duration_ms': 757.6999999880791, 'a...",0,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,4a129f35-d4be-4af0-8e13-71fed6a22215,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
3,bdd73bf5-b28e-49cb-bc57-69ce3e718435,bde7f3f1-e169-49de-9eb0-4ba5e74bd3af,db10ce98-b4a3-4208-8bef-5ef5c9ba9b3f,grid-memory,0.8.4,America/New_York,240,2023-10-18T20:22:59.193Z,2023-10-18T20:23:04.610Z,0,...,"[{'elapsed_duration_ms': 1114.6000000238419, '...",1,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,591c1aa6-e9a8-44a0-a3b8-6ac25180f831,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
4,dde6a316-13b4-4666-b694-231db4393559,d4d7c590-ea61-4fb6-8a9c-9adf071f5822,328f27b1-7901-4d91-b7a1-1e1c3614fe70,grid-memory,0.8.4,America/New_York,240,2023-10-18T20:26:15.301Z,2023-10-18T20:26:17.937Z,0,...,"[{'elapsed_duration_ms': 1750, 'action_type': ...",1,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 13;...,PSAC_PILOT,88291f12-8d32-4c8b-847a-7564ca854b2d,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
5,ed281e76-bf63-4a45-8d23-595138e2c0bc,6818a17e-4ddb-45c6-9260-2276fa0d9361,120bc122-2aa6-41af-a0bb-5fa2f613d8b9,grid-memory,0.8.4,America/New_York,240,2023-10-19T11:41:12.930Z,2023-10-19T11:41:16.006Z,0,...,"[{'elapsed_duration_ms': 2006, 'action_type': ...",3,False,{'userAgent': 'Mozilla/5.0 (iPhone; CPU iPhone...,PSAC_PILOT,689cafb4-3ac7-407f-9ac5-c1ae09cfd552,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
6,37e02bb2-b8bf-4bf8-81d1-e6f78682ab0b,f2744426-b62e-4f3f-a4b0-58cf53b4de4d,04662ca9-ce7c-4c01-aaa2-99c5794d0c71,grid-memory,0.8.4,America/New_York,240,2023-10-19T12:11:14.560Z,2023-10-19T12:11:30.138Z,0,...,"[{'elapsed_duration_ms': 1519, 'action_type': ...",3,False,{'userAgent': 'Mozilla/5.0 (iPhone; CPU iPhone...,PSAC_PILOT,c020c7c5-66e2-4e7c-a7f1-b18378445d33,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
7,4df68386-7a96-4999-bd79-c8cef4d150d0,773b6909-ba10-4cd5-93fb-0a08b6641079,f4a2340d-e8c9-4fe5-8661-601452a3616f,grid-memory,0.8.4,America/New_York,240,2023-10-19T14:29:30.283Z,2023-10-19T14:29:37.211Z,0,...,"[{'elapsed_duration_ms': 1519, 'action_type': ...",3,False,{'userAgent': 'Mozilla/5.0 (iPhone; CPU iPhone...,PSAC_PILOT,c92da645-e731-4b93-b1de-2240e7852b1d,None,e05ec3ec-bc42-40c4-91d9-7fd3d99983d8,None,None
8,cf745bd0-e32a-4085-af72-84dfed91989b,6c0fda4e-eeb1-42fe-9cc5-66b9e32ca826,baa10f49-00e0-4ef7-b212-743da736a6ca,grid-memory,0.8.4,America/New_York,240,2023-10-19T14:55:06.574Z,2023-10-19T14:55:08.911Z,0,...,"[{'elapsed_duration_ms': 1690, 'action_type': ...",3,False,{'userAgent': 'Mozilla/5.0 (iPhone; CPU iPhone...,PS

# Coming soon - this code as a pip installable package
<!-- pip install cookiecutter
cookiecutter https://github.com/waynerv/cookiecutter-pypackage.git -->